# Geostrophic adjustment and inertia-gravity waves
*MAQ - 32806, Chiel van Heerwaarden and Imme Benedict, 2017*

In this tutorial you will use the previously derived shallow water equations to study the concept of geostrophic adjustment (again pioneered by Carl-Gustaf Rossby). Geostrophic adjustment is the process that restores geostrophic and thermal wind balance after a disturbance, such as a big cloud system or a jet instability. In this tutorial, you will study how geostrophic adjustment leads to the formation of a low pressure system out of a disturbance, and you will learn about the role of inertia-gravity waves therein.
___

First, we load the required packages and set the physical constants.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
figsize_s = 6,4
figsize_l = 9,4
plt.rcParams.update({'legend.fontsize' : 'medium'})

In [ ]:
Omega = 7.29e-5    # Angular velocity of the earth.
g = 9.81           # Gravity constant.
a_earth = 6.371e6  # Earth radius.

___
# The shallow water equations for small perturbations
In this assignment, you are going to work with the linearized shallow water equations. This means that we assume that there is no background velocity, and that the velocities and height perturbations are small. Furthermore, to simplify the dynamics, we assume the Coriolis parameter $f$ to be a constant $f_0$. In addition to the derivation in class, we assume now that we have a flat bottom, with $z_b = 0$. Therefore, we can write $h$ directly into the conservation equations for horizontal momentum.

$$
\begin{eqnarray}
\nonumber
\dfrac{\partial u^\prime}{\partial t} & = & f_0 v^\prime - g\dfrac{\partial h^\prime}{\partial x}
\\
\nonumber
\dfrac{\partial v^\prime}{\partial t} & = & -f_0 u^\prime - g\dfrac{\partial h^\prime}{\partial y}
\\
\nonumber
\dfrac{\partial h^\prime}{\partial t} & = &
- \overline{h} \left( \dfrac{\partial u^\prime}{\partial x} +\dfrac{\partial v^\prime}{\partial y} \right)
\end{eqnarray}
$$

1. Explain the assumptions behind the equations above. Discuss in this explanation why can we neglect the advection terms if the perturbations are small, and why is $h$ replaced by $\overline{h}$ in the right-hand side of the third equation.
___

# The shallow water equations for our experiment
In our experiment, you are going to study the behavior of a two-dimensional system. We study an initially non-moving water layer in which we introduce a dip in the height that only varies with $x$. The governing equations are

$$
\begin{eqnarray}
\nonumber
\dfrac{\partial u^\prime}{\partial t} & = & f_0 v^\prime - g \dfrac{\partial h^\prime}{\partial x}
\\
\nonumber
\dfrac{\partial v^\prime}{\partial t} & = & -f_0 u^\prime
\\
\nonumber
\dfrac{\partial h^\prime}{\partial t} & = &
- \overline{h} \dfrac{\partial u^\prime}{\partial x}
\end{eqnarray}
$$

1. Explain why the governing equations can be written as above if the initial perturbation has no gradients in $y$.
___

# Setting up the experiment
Below, you can specify the setup of the experiment. The `f_multiplier` is initially set to `1` and can be adjusted later to make the earth spin faster or slower.

In [ ]:
f_multiplier = 1.
lat = 45.
f0 = f_multiplier * 2.*Omega*np.sin(np.deg2rad(lat))
L = a_earth * (2.*np.pi) * np.cos(np.deg2rad(lat))
L_dist = 1.e6
h_bar = 200.
h_prime = 0.1

With the conditions above, we can create our domain (ranging from `-L/2` to `L/2`). The initial condition of $u^{\prime}$, $v^{\prime}$ and $h^{\prime}$ are stored in variables `u0`, `v0` and `h0`.

In [ ]:
# Here we initialize the grid and the horizontal velocities.
nx = 1501
x = np.linspace(-L/2., L/2., nx)
dx = x[1]-x[0]
u0 = np.zeros(nx)
v0 = np.zeros(nx)

# The perturbation in h has the shape of a Gaussian (normal distribution).
x_sigma = L_dist/6.
h0 = np.exp(-x**2/(2.*x_sigma**2))
h0 = - h_prime*h0/np.max(h0)

In [ ]:
# This plot shows the shape of the initial perturbation
fig1 = plt.figure(figsize=figsize_l)
ax1 = plt.subplot(111)
ax1.plot(x, h0, 'C0-')
ax1.set_xlabel('x (m)')
ax1.set_ylabel('h\' (m)')
fig1.tight_layout()

1. Study the plot of the $h^{\prime}$ as a function of $x$. Is the perturbation really small? Compare its magnitude with that of $\overline{h}$.
2. Calculate the initial potential vorticity $q$ as a variable `q0` and plot it as a function of $x$.
3. Calculate the Rossby radius of deformation $\lambda_R \equiv \dfrac{\sqrt{g \overline{h}}}{f_0}$ for this experiment.
4. Replot $q$ and $h^{\prime}$ with the horizontal axis normalized by the Rossby radius $\lambda_R$ and normalize the axis between -8 and 8 $x/\lambda_R$. How large is the perturbation compared to the Rossby radius?

___
# Running the experiment
Now, you will run the model for `nt` timesteps with time increments of `dt` seconds. The output is saved every `ni` steps. The final result of $u^{\prime}$, $v^{\prime}$ and $h^{\prime}$ are stored in variables `u`, `v` and `h`. Below you find an animation that shows the time evolution that can aid you in interpreting what is happening.

In [ ]:
nt = 50000
ni = 500
dt = 10.

In [ ]:
# MODEL INTEGRATOR, THIS MIGHT TAKE A WHILE!
u = np.copy(u0)
v = np.copy(v0)
h = np.copy(h0)

h_out = np.zeros((nt//ni+1, nx))
h_out[0,:] = h0

for n in range(1,nt+1):
    u += dt * (   f0*v - g*np.gradient(h,dx) )
    v += dt * ( - f0*u )
    h += dt * ( - h_bar*np.gradient(u,dx) )
    
    if (n%ni == 0):
        h_out[n//ni,:] = h[:]

In [ ]:
# ANIMATION CREATION PHASE 1.
# Set up the figure, the axis, and the plot element we want to animate
fig1, ax1 = plt.subplots(figsize=figsize_l)
xmin, xmax = min(x), max(x)
hmin, hmax = np.min(h_out), np.max(h_out)
hmin -= 0.05*(hmax-hmin)
hmax += 0.05*(hmax-hmin)
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(hmin, hmax)
ax1.set_xlabel('x (m)')
ax1.set_ylabel('h (m)')
ax1.plot(x, h_out[0,:], 'b:')
line, = ax1.plot([], [], lw=2)
fig1.tight_layout()
plt.close()

In [ ]:
# ANIMATION CREATION PHASE 2
# Initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return (line,)

def animate(i):
    line.set_data(x, h_out[i,:])
    return (line,)

# Call the animator. Keyword blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig1, animate, init_func=init,
                               frames=h_out.shape[0], interval=40, blit=True)

HTML(anim.to_html5_video())

___
# Analysis of the experiment
Below you find the end result of the experiment. All three variables $h^{\prime}$, $u^{\prime}$ and $v^{\prime}$ are plotted.

In [ ]:
fig1 = plt.figure(figsize=figsize_l)
ax1 = plt.subplot(121)
ax1.plot(x, h , 'C0-')
ax1.plot(x, h0, 'C0:')
ax1.set_xlabel('x (m)')
ax1.set_ylabel('h\' (m)')
ax2 = plt.subplot(122)
ax2.plot(x, u, 'C1-', label='u')
ax2.plot(x, v, 'C2-', label='v')
ax2.set_xlabel('x (m)')
ax2.set_ylabel('u, v (m/s)')
ax2.legend(loc=0, frameon=False);
fig1.tight_layout()

1. What do you see in the figure above? Is this what you expected? Why does the surface not become flat?
2. Replot the figure above, but normalize $x$ again with $\lambda_R$ and limit the range between -8 and 8. Within how many Rossby radii is the initial perturbation contained?
3. Calculate the potential vorticity at the end of the simulation and compare it to the initial value. How has it changed?
4. Rerun the experiment for an earth that spins five times faster (set `f_multiplier=5.`) and for one that spins five times slower (set `f_multiplier=0.2`). What do you observe?
5. What is the influence of the rotation rate on the phase velocity and group velocity of the waves? Can you link your observations to the dispersion relation of inertia-gravity waves?
___

#  Geostrophic adjustment
With the help of the potential vorticity, it is possible for this experiment to predict the final steady state out of the initial conditions. The formulation of the steady state is this specific experiment is

$$
\begin{eqnarray}
\nonumber
0 & = & f_0 v^\prime - g \dfrac{\partial h^\prime}{\partial x}
\\
\nonumber
0 & = & -f_0 u^\prime
\\
\nonumber
0 & = &
- \overline{h} \dfrac{\partial u^\prime}{\partial x}
\end{eqnarray}
$$

We know that throughout the geostrophic adjustment the potential vorticity has not changed, thus

$$
q = q_0
$$
___
1. Why is the process that you have modelled called geostrophic adjustment? TIPS: Think about how to define geostrophic wind speed in the shallow water equations. Don't forget that $\dfrac{\partial h^{\prime}}{\partial y} = 0$ in this experiment.
2. Derive the equation that relates the final height $h^{\prime}$ to the initial potential vorticity $q_0$. This is a differential equation, you do not have to solve it.
___

Below you find a solver for the final state as a function of the initial potential vorticity.

In [ ]:
# MAKE SURE THAT YOU HAVE CALCULATED THE INITIAL POTENTIAL VORTICITY
# AS A VARIABLE q0
q0 = f0 / (h_bar + h0)

def calc_steady_state():
    c1 = g/(f0*dx**2)

    # Matrix solver
    M = np.zeros((nx, nx))

    for i in range(1,nx-1):
        M[i,i-1] = c1
        M[i,i  ] = -(2.*c1 + q0[i])
        M[i,i+1] = c1

    M[ 0, 0] = 1.
    M[-1,-1] = 1.

    h_mat = -1.*np.ones(nx)*f0 + q0*h_bar
    h_mat[ 0] = 0
    h_mat[-1] = 0

    h = np.linalg.inv(M).dot(h_mat)
    v = g/f0*np.gradient(h, dx)
    
    return h, v

h_inv, v_inv = calc_steady_state()

In [ ]:
fig1 = plt.figure(figsize=figsize_l)
ax1 = plt.subplot(121)
# ADD HERE THE SOLUTION FOR H (h_inv)
ax1.plot(x, h, 'k--', label='model')
ax1.set_xlabel('x (m)')
ax1.set_ylabel('h\' (m)')
ax1.legend(loc=0, frameon=False)
ax2 = plt.subplot(122)
# ADD HERE THE SOLUTION FOR V (v_inv)
ax2.plot(x, v, 'k--', label='model')
ax2.set_xlabel('x (m)')
ax2.set_ylabel('v (m/s)')
ax2.legend(loc=0, frameon=False);
fig1.tight_layout()

1. Add the solution acquired from the matrix inversion to the plot above, and normalize $x$ with the Rossby radius. Is the result the same? If not, explain the differences.
2. Explain the pattern in the final solution for $v$. What are the similarities between this pattern and a zonal cross section through the center of a low pressure system in the lower troposphere? 

___
End of tutorial.
___